In [1]:
!pip uninstall torch -y
!pip install torch==2.0.0

Found existing installation: torch 2.0.0
Uninstalling torch-2.0.0:
  Successfully uninstalled torch-2.0.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torch-2.0.0-cp39-cp39-manylinux1_x86_64.whl (619.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 2.0.0 which is incompatible.
torchtext 0.14.1 requires torch==1.13.1, but you have torch 2.0.0 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 2.0.0 which is incompatible.
fastai 2.7.11 requires torch<1.14,>=1.7, but you have torch 2.0.0 which is incompatible.


In [2]:
!rm -rf restaurant_true
!rm -rf restaurant

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip /content/drive/MyDrive/SWDE/Restaurant/restaurant_true.zip

Archive:  /content/drive/MyDrive/SWDE/Restaurant/restaurant_true.zip
   creating: restaurant_true/
  inflating: restaurant_true/restaurant-fodors-address.txt  
  inflating: restaurant_true/restaurant-fodors-cuisine.txt  
  inflating: restaurant_true/restaurant-fodors-name.txt  
  inflating: restaurant_true/restaurant-fodors-phone.txt  
  inflating: restaurant_true/restaurant-frommers-address.txt  
  inflating: restaurant_true/restaurant-frommers-cuisine.txt  
  inflating: restaurant_true/restaurant-frommers-name.txt  
  inflating: restaurant_true/restaurant-frommers-phone.txt  
  inflating: restaurant_true/restaurant-gayot-address.txt  
  inflating: restaurant_true/restaurant-gayot-cuisine.txt  
  inflating: restaurant_true/restaurant-gayot-name.txt  
  inflating: restaurant_true/restaurant-gayot-phone.txt  
  inflating: restaurant_true/restaurant-opentable-address.txt  
  inflating: restaurant_true/restaurant-opentable-cuisine.txt  
  inflating: restaurant_true/restaurant-opentable-na

In [5]:
!unzip /content/drive/MyDrive/SWDE/Restaurant/restaurant.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: restaurant/restaurant-opentable(2000)/0893.htm  
  inflating: restaurant/restaurant-opentable(2000)/0391.htm  
  inflating: restaurant/restaurant-opentable(2000)/1538.htm  
  inflating: restaurant/restaurant-opentable(2000)/1017.htm  
  inflating: restaurant/restaurant-opentable(2000)/1412.htm  
  inflating: restaurant/restaurant-opentable(2000)/1339.htm  
  inflating: restaurant/restaurant-opentable(2000)/1725.htm  
  inflating: restaurant/restaurant-opentable(2000)/1030.htm  
  inflating: restaurant/restaurant-opentable(2000)/0580.htm  
  inflating: restaurant/restaurant-opentable(2000)/0673.htm  
  inflating: restaurant/restaurant-opentable(2000)/1284.htm  
  inflating: restaurant/restaurant-opentable(2000)/1682.htm  
  inflating: restaurant/restaurant-opentable(2000)/1881.htm  
  inflating: restaurant/restaurant-opentable(2000)/1511.htm  
  inflating: restaurant/restaurant-opentable(2000)/0908.htm  
  

In [6]:
!pip install -q git+https://github.com/huggingface/transformers.git  # Transormers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [7]:
!pip install -q evaluate seqeval  # For HuggingFace metrics

Let's show labels:

In [8]:
basic_labels = ["name", "address", "phone", "cuisine", "other"]

labels_with_ids = {label:ind for ind, label in enumerate(basic_labels)}
ids_with_labels = {ind:label for label, ind in labels_with_ids.items()}

In [9]:
print(labels_with_ids)
print("-----")
print(ids_with_labels)

{'name': 0, 'address': 1, 'phone': 2, 'cuisine': 3, 'other': 4}
-----
{0: 'name', 1: 'address', 2: 'phone', 3: 'cuisine', 4: 'other'}


In [10]:
from os import listdir

restaurant_dir = listdir("/content/restaurant")
# print(restaurant_dir)

Now we are going to extract features for model

In [129]:
labels = []
pathname = lambda label: f"/content/restaurant_true/restaurant-urbanspoon-{label}.txt"

with open(pathname("name"), 'r') as names, open(pathname("address"), 'r') as addresses, open(pathname("cuisine"), 'r') as cuisines, open(pathname("phone"), 'r') as phones:
     names.readline()
     names.readline()
     addresses.readline()
     addresses.readline()
     cuisines.readline()
     cuisines.readline()
     phones.readline()
     phones.readline()
     for i in range(2000):
        index = f"{i: 05d}"
        _, _, *name = names.readline().split()
        name = " ".join(name)
        _, _, *address = addresses.readline().split()
        # address = " ".join(address)
        _, _, *cuisine = cuisines.readline().split()
        # cuisine = " ".join(cuisine)
        _, _, *phone = phones.readline().split()
        phone = " ".join(phone)
        labels.append(
            {
                "name": name,
                "address": address,
                "phone": phone,
                "cuisine": cuisine
            }
        )

# print(len(labels))
# print(labels[:10])

<h1> Questions:

Question: how are we going to split dataset into train and test datasets? \
~~Question: do we use MarkupLM-base?~~ \
Question: do we need different types of web-sites for training (one aggregator for every 2000)? \
~~Question: why is there [0] in ~~
~```
...
~for node_text in encoding['nodes'][0]:~
...
```~``` \
Question: can I use other optimizers and hyperparametres? \
~~Question: as I understood Transformers are self-supervised, so how do we test them for true prediction? ~~\
Question: too many duplicate code??? \


Была проблема с кухней и адресом. Исправил так:

In [87]:
from transformers import MarkupLMFeatureExtractor
feature_extractor = MarkupLMFeatureExtractor()

data = []

for i in range(50):
    with open(f"/content/restaurant/restaurant-urbanspoon(2000)/{i:04d}.htm") as html_page:
        html_code = html_page.read()
    encoding = feature_extractor(html_code)
    node_labels = [[]]
    for node_text in encoding['nodes'][0]:
        # do this with match! ( match <expr>: )
        if node_text == labels[i]["name"]:
            node_labels[0].append(labels_with_ids["name"])
        elif node_text in labels[i]["address"]:
            node_labels[0].append(labels_with_ids["address"])
        elif node_text == labels[i]["phone"]:
            node_labels[0].append(labels_with_ids["phone"])
        elif node_text in labels[i]["cuisine"]:
            node_labels[0].append(labels_with_ids["cuisine"])
        else:
            node_labels[0].append(labels_with_ids["other"])
        
    data.append({'nodes': encoding['nodes'],
                 'xpaths': encoding['xpaths'],
                 'node_labels': node_labels})

In [88]:
print(len(data[0]['nodes'][0]))
print(len(data[0]['xpaths'][0]))
print(len(data[0]['node_labels'][0]))

681
681
681


In [89]:
# data[0] # This suits for 0000.htm in urbanspoon

In [90]:
from torch.utils.data import Dataset

class MarkupLMDataset(Dataset):
    """Dataset for token classification with MarkupLM."""

    def __init__(self, data, processor=None, max_length=512):
        self.data = data
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # first, get nodes, xpaths and node labels
        item = self.data[idx] # May be to delete!
        nodes, xpaths, node_labels = item['nodes'], item['xpaths'], item['node_labels']

        # provide to processor
        encoding = self.processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels, padding="max_length",
                                  max_length=self.max_length, return_tensors="pt", truncation=True)

        # remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}

        return encoding

In [91]:
from transformers import MarkupLMProcessor

processor = MarkupLMProcessor.from_pretrained("microsoft/markuplm-base")
processor.parse_html = False

In [92]:
dataset = MarkupLMDataset(data=data, processor=processor, max_length=512)

In [93]:
example = dataset[1]
for k,v in example.items():
  print(k,v.shape)

input_ids torch.Size([512])
token_type_ids torch.Size([512])
attention_mask torch.Size([512])
xpath_tags_seq torch.Size([512, 50])
xpath_subs_seq torch.Size([512, 50])
labels torch.Size([512])


In [94]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [95]:
# for batch in dataloader:
#   print(batch)

In [96]:
from transformers import MarkupLMForTokenClassification

model = MarkupLMForTokenClassification.from_pretrained("microsoft/markuplm-base", id2label=ids_with_labels, label2id=labels_with_ids)

Some weights of the model checkpoint at microsoft/markuplm-base were not used when initializing MarkupLMForTokenClassification: ['nrp_cls.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.bias', 'markuplm.pooler.dense.weight', 'cls.predictions.transform.dense.bias', 'nrp_cls.decoder.bias', 'nrp_cls.dense.bias', 'nrp_cls.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'ptc_cls.weight', 'ptc_cls.bias', 'markuplm.pooler.dense.bias', 'cls.predictions.transform.dense.weight', 'nrp_cls.dense.weight', 'cls.predictions.bias', 'nrp_cls.decoder.weight']
- This IS expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MarkupLMForTokenClassification from the checkpoint of a model that 

In [97]:
label_list = ["B-" + x for x in list(ids_with_labels.values())]
label_list

['B-name', 'B-address', 'B-phone', 'B-cuisine', 'B-other']

In [98]:
import evaluate

# Metric
metric = evaluate.load("seqeval")

def get_labels(predictions, references):
    # Transform predictions and references tensos to numpy arrays
    if device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()
    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

def compute_metrics(return_entity_level_metrics=True):
    results = metric.compute()
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [99]:
# for batch in dataloader:
#   print(batch)

In [100]:
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm

optimizer = AdamW(model.parameters(), lr=5e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

model.train()

for epoch in range(20):  # loop over the dataset multiple times
    for batch in dataloader:
        print("-----BATCH-----")
        # get the inputs;
        inputs = {k:v.to(device) for k,v in batch.items()}

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**inputs)

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print("Loss:", loss.item())

        predictions = outputs.logits.argmax(dim=-1)
        true_labels = batch["labels"]
        preds, refs = get_labels(predictions, true_labels)
        metric.add_batch(
            predictions=preds,
            references=refs,
        )

    eval_metric = compute_metrics()
    print(f"Epoch {epoch + 1}:", eval_metric)

-----BATCH-----
Loss: 1.3425265550613403
-----BATCH-----
Loss: 0.6036854386329651
-----BATCH-----
Loss: 0.35575422644615173
-----BATCH-----
Loss: 0.33483079075813293
Epoch 1: {'address_precision': 0.0, 'address_recall': 0.0, 'address_f1': 0.0, 'address_number': 243, 'cuisine_precision': 0.012059369202226345, 'cuisine_recall': 0.14130434782608695, 'cuisine_f1': 0.022222222222222223, 'cuisine_number': 92, 'name_precision': 0.0, 'name_recall': 0.0, 'name_f1': 0.0, 'name_number': 43, 'other_precision': 0.9266327396098388, 'other_recall': 0.8070175438596491, 'other_f1': 0.8626986477149344, 'other_number': 5415, 'phone_precision': 0.0, 'phone_recall': 0.0, 'phone_f1': 0.0, 'phone_number': 50, 'overall_precision': 0.7501283587198357, 'overall_recall': 0.7501283587198357, 'overall_f1': 0.7501283587198357, 'overall_accuracy': 0.7501283587198357}
-----BATCH-----
Loss: 0.32479190826416016
-----BATCH-----
Loss: 0.2643498480319977
-----BATCH-----
Loss: 0.22684286534786224
-----BATCH-----
Loss: 0.16

/usr/local/lib/python3.9/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2: {'address_precision': 1.0, 'address_recall': 0.037037037037037035, 'address_f1': 0.07142857142857142, 'address_number': 243, 'cuisine_precision': 0.0, 'cuisine_recall': 0.0, 'cuisine_f1': 0.0, 'cuisine_number': 92, 'name_precision': 0.0, 'name_recall': 0.0, 'name_f1': 0.0, 'name_number': 43, 'other_precision': 0.9281796366129585, 'other_recall': 1.0, 'other_f1': 0.9627522446439685, 'other_number': 5415, 'phone_precision': 0.0, 'phone_recall': 0.0, 'phone_f1': 0.0, 'phone_number': 50, 'overall_precision': 0.9282902618517884, 'overall_recall': 0.9282902618517884, 'overall_f1': 0.9282902618517884, 'overall_accuracy': 0.9282902618517884}
-----BATCH-----
Loss: 0.170380100607872
-----BATCH-----
Loss: 0.1633291095495224
-----BATCH-----
Loss: 0.14730802178382874
-----BATCH-----
Loss: 0.17020277678966522
Epoch 3: {'address_precision': 0.8414634146341463, 'address_recall': 0.8518518518518519, 'address_f1': 0.8466257668711655, 'address_number': 243, 'cuisine_precision': 0.0, 'cuisine_rec

KeyboardInterrupt: ignored

In [132]:
labels

[{'name': 'Frontier Restaurant',
  'address': ['2400', 'Central', 'Ave', 'SE', 'Albuquerque', 'NM', '87106'],
  'phone': '(505) 266-0550',
  'cuisine': ['Diner', 'New', 'Mexican']},
 {'name': "Saggio's",
  'address': ['107', 'Cornell', 'Dr', 'SE', 'Albuquerque', 'NM', '87106'],
  'phone': '(505) 255-5454',
  'cuisine': ['Pizza', 'Italian']},
 {'name': 'Tucanos Brazilian Grill',
  'address': ['110', 'Central', 'Ave', 'SW', 'Albuquerque', 'NM', '87102'],
  'phone': '(505) 246-9900',
  'cuisine': ['Brazilian']},
 {'name': 'Dog House Drive in',
  'address': ['1216', 'Central', 'Ave', 'SW', 'Albuquerque', 'NM', '87102'],
  'phone': '(505) 243-1019',
  'cuisine': ['American', 'Hot', 'Dogs']},
 {'name': 'India Palace',
  'address': ['4410', 'Wyoming', 'Blvd', 'NE', 'Albuquerque', 'NM', '87111'],
  'phone': '(505) 271-5009',
  'cuisine': ['Indian']},
 {'name': 'Namaste',
  'address': ['1520', 'Deborah', 'Rd.', 'SE', 'Rio', 'Rancho', 'NM', '87124'],
  'phone': '(505) 896-3126',
  'cuisine': ['I

In [158]:
new_data = []

i = 60
with open(f"/content/restaurant/restaurant-urbanspoon(2000)/0060.htm") as html_page:
    html_code = html_page.read()
    encoding = feature_extractor(html_code)
    node_labels = [[]]
    for node_text in encoding['nodes'][0]:
        # do this with match! ( match <expr>: )
        if node_text == labels[i]["name"]:
            node_labels[0].append(labels_with_ids["name"])
        elif node_text == labels[i]["address"]:
            node_labels[0].append(labels_with_ids["address"])
        elif node_text == labels[i]["phone"]:
            node_labels[0].append(labels_with_ids["phone"])
        elif node_text == labels[i]["cuisine"]:
            node_labels[0].append(labels_with_ids["cuisine"])
        else:
            node_labels[0].append(labels_with_ids["other"])
        
    new_data.append({'nodes': encoding['nodes'],
                     'xpaths': encoding['xpaths'],
                     'node_labels': node_labels})

In [159]:
# new_data

In [160]:
nodes = new_data[0]['nodes']
xpaths = new_data[0]['xpaths']
node_labels = new_data[0]['node_labels']
print("Nodes:", nodes)
print("Xpaths:", xpaths)

Nodes: [["Guero's Taco Bar - South Congress - SoCo - Austin | Urbanspoon", 'Sign In', 'Choose a city:', 'Austin', 'Dallas', 'Houston', 'Los Angeles', 'New York', 'San Antonio', 'Texas', 'More Cities...', 'Urbanspoon Austin', 'Put this on your site', 'See all 6 photos', 'See all', '37', 'reviews', 'Get our newsletter', 'Subscribe to our monthly newsletter to get the scoop on Austin restaurants.', 'See details', 'Home', '>', 'Austin', '>', 'South Congress - SoCo', '>', 'Is this your restaurant?', "Guero's Taco Bar", '(512) 447-7688', 'South Congress - SoCo', '1412 S Congress Ave', 'Austin', ',', 'TX', '78704', 'www.guerostacobar.com', 'Send to phone', '533 people have voted', '78%', 'like it', 'Add your vote', 'I like it', "I don't", 'Add to your wishlist', 'Write a review', 'Send to a friend', 'Edit restaurant info', "Menu for Guero's Taco Bar", 'm', 't', 'w', 't', 'f', 's', 's', 'breakfast', 'x', 'x', 'lunch', 'x', 'x', 'x', 'x', 'x', 'x', 'x', 'dinner', 'x', 'x', 'x', 'x', 'x', 'x', '

In [161]:
encoding = processor(nodes=nodes, xpaths=xpaths, node_labels=node_labels, return_offsets_mapping=True, return_tensors="pt", truncation=True).to(device)
for k,v in encoding.items():
  print(k,v.shape)

input_ids torch.Size([1, 512])
token_type_ids torch.Size([1, 512])
attention_mask torch.Size([1, 512])
offset_mapping torch.Size([1, 512, 2])
xpath_tags_seq torch.Size([1, 512, 50])
xpath_subs_seq torch.Size([1, 512, 50])
labels torch.Size([1, 512])


In [162]:
for k,v in encoding.items():
  print(k,v.shape)

input_ids torch.Size([1, 512])
token_type_ids torch.Size([1, 512])
attention_mask torch.Size([1, 512])
offset_mapping torch.Size([1, 512, 2])
xpath_tags_seq torch.Size([1, 512, 50])
xpath_subs_seq torch.Size([1, 512, 50])
labels torch.Size([1, 512])


In [163]:
print(torch.__version__)

2.0.0+cu117


In [164]:
offset_mapping = encoding.pop("offset_mapping")
encoding_labels = encoding.pop("labels")

# forward pass
with torch.no_grad():
  outputs = model(**encoding)

In [165]:
labels

[{'name': 'Frontier Restaurant',
  'address': ['2400', 'Central', 'Ave', 'SE', 'Albuquerque', 'NM', '87106'],
  'phone': '(505) 266-0550',
  'cuisine': ['Diner', 'New', 'Mexican']},
 {'name': "Saggio's",
  'address': ['107', 'Cornell', 'Dr', 'SE', 'Albuquerque', 'NM', '87106'],
  'phone': '(505) 255-5454',
  'cuisine': ['Pizza', 'Italian']},
 {'name': 'Tucanos Brazilian Grill',
  'address': ['110', 'Central', 'Ave', 'SW', 'Albuquerque', 'NM', '87102'],
  'phone': '(505) 246-9900',
  'cuisine': ['Brazilian']},
 {'name': 'Dog House Drive in',
  'address': ['1216', 'Central', 'Ave', 'SW', 'Albuquerque', 'NM', '87102'],
  'phone': '(505) 243-1019',
  'cuisine': ['American', 'Hot', 'Dogs']},
 {'name': 'India Palace',
  'address': ['4410', 'Wyoming', 'Blvd', 'NE', 'Albuquerque', 'NM', '87111'],
  'phone': '(505) 271-5009',
  'cuisine': ['Indian']},
 {'name': 'Namaste',
  'address': ['1520', 'Deborah', 'Rd.', 'SE', 'Rio', 'Rancho', 'NM', '87124'],
  'phone': '(505) 896-3126',
  'cuisine': ['I

In [166]:
predictions = outputs.logits.argmax(-1)
print(predictions)

tensor([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 2,
         2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 1, 1, 1, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4

In [167]:
results = {"Node": [], "Predicted": [], "Ground truth": []}

for pred_id, word_id, offset, label_id in zip(predictions[0].tolist(), encoding.word_ids(0), offset_mapping[0].tolist(), encoding_labels[0].tolist()):
  if word_id is not None and offset[0] == 0:
    print(f"Node: {nodes[0][word_id]}")
    print(f"Predicted: {ids_with_labels[pred_id]}")
    print(f"Ground truth: {ids_with_labels[label_id]}")
    print("----------")
    results["Node"].append(nodes[0][word_id])
    results["Predicted"].append(ids_with_labels[pred_id])
    results["Ground truth"].append(ids_with_labels[label_id])

Node: Guero's Taco Bar - South Congress - SoCo - Austin | Urbanspoon
Predicted: other
Ground truth: other
----------
Node: Sign In
Predicted: other
Ground truth: other
----------
Node: Choose a city:
Predicted: other
Ground truth: other
----------
Node: Austin
Predicted: other
Ground truth: other
----------
Node: Dallas
Predicted: other
Ground truth: other
----------
Node: Houston
Predicted: other
Ground truth: other
----------
Node: Los Angeles
Predicted: other
Ground truth: other
----------
Node: New York
Predicted: other
Ground truth: other
----------
Node: San Antonio
Predicted: other
Ground truth: other
----------
Node: Texas
Predicted: other
Ground truth: other
----------
Node: More Cities...
Predicted: other
Ground truth: other
----------
Node: Urbanspoon Austin
Predicted: other
Ground truth: other
----------
Node: Put this on your site
Predicted: other
Ground truth: other
----------
Node: See all 6 photos
Predicted: other
Ground truth: other
----------
Node: See all
Predicted: 

In [168]:
labels

[{'name': 'Frontier Restaurant',
  'address': ['2400', 'Central', 'Ave', 'SE', 'Albuquerque', 'NM', '87106'],
  'phone': '(505) 266-0550',
  'cuisine': ['Diner', 'New', 'Mexican']},
 {'name': "Saggio's",
  'address': ['107', 'Cornell', 'Dr', 'SE', 'Albuquerque', 'NM', '87106'],
  'phone': '(505) 255-5454',
  'cuisine': ['Pizza', 'Italian']},
 {'name': 'Tucanos Brazilian Grill',
  'address': ['110', 'Central', 'Ave', 'SW', 'Albuquerque', 'NM', '87102'],
  'phone': '(505) 246-9900',
  'cuisine': ['Brazilian']},
 {'name': 'Dog House Drive in',
  'address': ['1216', 'Central', 'Ave', 'SW', 'Albuquerque', 'NM', '87102'],
  'phone': '(505) 243-1019',
  'cuisine': ['American', 'Hot', 'Dogs']},
 {'name': 'India Palace',
  'address': ['4410', 'Wyoming', 'Blvd', 'NE', 'Albuquerque', 'NM', '87111'],
  'phone': '(505) 271-5009',
  'cuisine': ['Indian']},
 {'name': 'Namaste',
  'address': ['1520', 'Deborah', 'Rd.', 'SE', 'Rio', 'Rancho', 'NM', '87124'],
  'phone': '(505) 896-3126',
  'cuisine': ['I